In [1]:
import requests
import pandas as pd
import geopandas as gpd
import datetime
from shapely import wkt, wkb
import polyline
from shapely.geometry import LineString, Point
# skipping for now - I'll just do the upload later
# import boto3
import numpy as np

In [2]:
def Random_Points_in_Bounds(polygon, number):   
    minx, miny, maxx, maxy = polygon.bounds
    x = np.random.uniform( minx, maxx, number )
    y = np.random.uniform( miny, maxy, number )
    return x, y

def generate_points_within_tm_boundary(tm_boundary, trimet_crs):
    '''
    generate 2 points and make sure they are > 1 mile apart
    '''
    
    #the length of the line connecting the two points is the 
    #distance between them (as crow flies)
    dist_btw_points = 0
    while dist_btw_points < 1:
        x, y = Random_Points_in_Bounds(tm_boundary.unary_union, 2)
        dist_btw_points = LineString(list(zip(x,y))).length/5280

    df = pd.DataFrame()
    df['points'] = list(zip(x,y))
    df['points'] = df['points'].apply(Point)
    gdf_points = gpd.GeoDataFrame(df,crs=trimet_crs ,geometry='points')
    gdf_points_reproject = gdf_points.to_crs("EPSG:4326")

    return (gdf_points_reproject, dist_btw_points)

def call_planner(fromPlace, toPlace):
    ''' 
    fromPlace = "lat, lon"
    toPlace = "lat,lon"
    '''
    date = datetime.datetime.now().strftime("%Y-%m-%d")
    base_url = "https://maps.trimet.org/otp_mod/plan"
    time="12:00"
    mode="WALK,BUS,TRAM,RAIL,GONDOLA"
    maxWalkDistance=1609
    walkSpeed=1.34
    numItineraries=3
    r = requests.get(url=base_url, params={'fromPlace':fromPlace, 'toPlace':toPlace, 'date':date, 'time':time
                                    ,'mode':mode, 'maxWalkDistance':maxWalkDistance, 'walkSpeed':walkSpeed
                                    ,'numItineraries':numItineraries})
    assert(r.status_code==200)
    return r.json()

def decode_create_leg_line(encoded_linestring):
    '''google polyline encoded linestring'''
    reformatted_coords = []
    original_coords = polyline.decode(encoded_linestring)
    for coord in original_coords:
        reformatted_coords.append((coord[1],coord[0]))
    return LineString(reformatted_coords)

def get_itinerary_paths(planner_response):
    ''' 
    planner_response = json_response from TriMet trip planner
    '''
    itineraries_df = pd.DataFrame()
    for itin_idx, itinerary in enumerate(planner_response['plan']['itineraries']):
        totalTime = itinerary['duration']
        walkTime = itinerary['walkTime']
        transitTime = itinerary['transitTime']
        waitingTime = itinerary['waitingTime']
        walkDistance = itinerary['walkDistance']
        for leg_idx, leg in enumerate(itinerary['legs']):
            route_id = leg.get('routeId','').split(":")[-1]
            mode = leg['mode']
            fromName = leg['from']['name']
            toName = leg['to']['name']
            fromStopCode = leg['from'].get('stopCode','')
            toStopCode = leg['to'].get('stopCode','')
            legGeometry = decode_create_leg_line(leg['legGeometry']['points'])
            leg_df = pd.DataFrame([[itin_idx, totalTime, walkTime, transitTime, waitingTime, walkDistance,
                                   leg_idx, route_id, mode, fromStopCode, fromName, toStopCode, toName,
                                   legGeometry]],
                                   columns=['itin_idx', 'totalTime', 'walkTime', 'transitTime', 'waitingTime', 'walkDistance',
                                   'leg_id', 'route_id', 'mode', 'fromStopCode', 'fromName', 'toStopCode', 'toName',
                                   'legGeometry'])
            itineraries_df = pd.concat([itineraries_df,leg_df])
    return itineraries_df

## basic plan generate random points within trimet service area
**Steps**
1. get TriMet service area
2. generate 2 points
3. check that points are not within 1 mile of each other (as the crow flies)

## advanced plan generate random point within 2 portland block groups
**Steps**
1. Download block groups
2. get TriMet service area
3. generate points within two separate block groups - advanced no adjacent?

In [3]:
tm_boundary = gpd.read_file("gis/export/tm_boundary.shp")
tm_boundary.head(3)

,area_sq_mi,acres,geometry
0,533,341554,"POLYGON ((7713384.554 639002.575, 7713365.501 ..."


In [4]:
trimet_crs = tm_boundary.crs

In [5]:
trimet_crs

<Derived Projected CRS: EPSG:2913>
Name: NAD83(HARN) / Oregon North (ft)
Axis Info [cartesian]:
- X[east]: Easting (foot)
- Y[north]: Northing (foot)
Area of Use:
- name: United States (USA) - Oregon - counties of Baker; Benton; Clackamas; Clatsop; Columbia; Gilliam; Grant; Hood River; Jefferson; Lincoln; Linn; Marion; Morrow; Multnomah; Polk; Sherman; Tillamook; Umatilla; Union; Wallowa; Wasco; Washington; Wheeler; Yamhill.
- bounds: (-124.17, 43.95, -116.47, 46.26)
Coordinate Operation:
- name: SPCS83 Oregon North zone (International feet)
- method: Lambert Conic Conformal (2SP)
Datum: NAD83 (High Accuracy Reference Network)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [43]:
gdf_points, dist_btw_points = generate_points_within_tm_boundary(tm_boundary, trimet_crs)

In [44]:
gdf_points.explore()

In [30]:
gdf_points['points_str'] = gdf_points['points'].apply(lambda x: f"{round(x.x,6)}, {round(x.y,6)}")

In [34]:
fromplace, toplace = gdf_points['points_str'].to_numpy()

In [36]:
json_content = call_planner(fromplace, toplace)

In [40]:
json_content['error']

{'id': 404,
 'msg': 'No trip found. There may be no transit service within the maximum specified distance or at the specified time, or your start or end point might not be safely accessible.',
 'message': 'PATH_NOT_FOUND',
 'noPath': True}

In [37]:
itineraries_df = get_itinerary_paths(json_content)

KeyError: 'plan'

In [ ]:
itineraries_df.head(2)

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry
0,0,2409,713,1283,413,1007.445463,0,,WALK,,Origin,14243,SE Division & 20th Ave,"LINESTRING (-122.64587 45.50428, -122.64587 45..."
0,0,2409,713,1283,413,1007.445463,1,2,BUS,14243,SE Division & 20th Ave,13773,OMSI/SE Water,"LINESTRING (-122.64601 45.50482, -122.64607 45..."


In [ ]:
itineraries_gdf = gpd.GeoDataFrame(itineraries_df, crs="4326", geometry="legGeometry")
itineraries_gdf.head(2)

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry
0,0,2409,713,1283,413,1007.445463,0,,WALK,,Origin,14243,SE Division & 20th Ave,"LINESTRING (-122.64587 45.50428, -122.64587 45..."
0,0,2409,713,1283,413,1007.445463,1,2,BUS,14243,SE Division & 20th Ave,13773,OMSI/SE Water,"LINESTRING (-122.64601 45.50482, -122.64607 45..."


In [ ]:
itineraries_gdf[itineraries_gdf['itin_idx']==1].explore(column='route_id', tiles='CartoDB positron')

In [ ]:
itineraries_gdf.to_file("gis/export/itineraries.geojson", driver="GeoJSON")

In [ ]:
client = boto3.client("s3")
client.upload_file("gis/export/itineraries.geojson", "meysohn-sandbox", "trimet_trip_planner/itineraries.geojson",ExtraArgs={'ACL':'public-read'})

In [ ]:
# fromplace = "45.504286,-122.646199"
# toplace = "45.542102,-122.664715"

to_from_df = 